In [6]:
import pandas as pd
import numpy as np
from netalign.approx_isorank.io_utils import compute_adjacency, compute_pairs
from netalign.approx_isorank.isorank_compute import compute_isorank, compute_greedy_assignment, pair_acc
from netalign.approx_isorank.pair_evaluations import compute_edge_correctness, semantic_sim, symmetric_substructure, lccs
from numpy.linalg import norm
import json
import scipy.io.loadmat as loadmat

ModuleNotFoundError: No module named 'scipy.io'

In [ ]:
!which python

/cluster/tufts/rt/software/jupyter/bin/python


In [ ]:
sp1 = "bakers"
sp2 = "rat"

finalloc = f"temp-data/{sp1}-{sp2}.mat"

MAT = loadmat(finalloc)[f"{sp1}_{sp2}"]

In [47]:
pairs = compute_greedy_assignment(MAT, 1000)

/home/kapil/.conda/envs/main/lib/python3.9/site-packages/scipy/sparse/_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [48]:
dataloc = "../../data/intact/"
df1 = pd.read_csv(f"{dataloc}/{sp1}.s.tsv", header = None, sep = "\t")
df2 = pd.read_csv(f"{dataloc}/{sp2}.s.tsv", header = None, sep = "\t")

df1

,0,1
0,P08539,P18851
1,P11433,P18852
2,P11433,P21268
3,P11433,P18851
4,P18851,P18852
...,...,...
109681,Q07821,Q12425
109682,P47158,Q12425
109683,P47158,Q07821
109684,P47158,Q12425


In [49]:
with open(f"temp-data/{sp1}.json") as ij1, open(f"temp-data/{sp2}.json") as ij2:
    nm1 = json.load(ij1)
    nm2 = json.load(ij2)

dfa1 = df1.copy()
dfa2 = df2.copy()

dfa1[0] = dfa1[0].apply(lambda x : nm1[x])
dfa1[1] = dfa1[1].apply(lambda x : nm1[x])


dfa2[0] = dfa2[0].apply(lambda x : nm2[x])
dfa2[1] = dfa2[1].apply(lambda x : nm2[x])

In [50]:
pairs = [(i, j[0, 0]) for i, j in pairs]

In [52]:
ec = compute_edge_correctness(pairs, dfa1, dfa2)
ec

0.013049192639879834

In [53]:
symmetric_substructure(pairs, dfa1, dfa2)

0.011878311399760725

In [54]:
lccs(pairs, dfa1, dfa2)

22

In [55]:
rnA1 = {v: k for k, v in nm1.items()}
rnA2 = {v: k for k, v in nm2.items()}

annot1 = f"../../data/go/{sp1}.output.mapping.gaf"
annot2 = f"../../data/go/{sp2}.output.mapping.gaf"
FC={}
pairs_ = [(rnA1[p], rnA2[q]) for p, q in pairs]
print(pairs_)

[('P22137', 'P11442'), ('P07259', 'P07756'), ('P32327', 'P52873'), ('P32324', 'P05197'), ('P25694', 'P46462'), ('P19414', 'Q9ER34'), ('P06786', 'P41516'), ('P07245', 'P27653'), ('P39109', 'Q63120'), ('P07806', 'Q04462'), ('P36022', 'P38650'), ('P09435', 'P63018'), ('P10591', 'P55063'), ('P22202', 'P14659'), ('P32600', 'P42346'), ('P0CS90', 'P48721'), ('P16474', 'P06761'), ('Q00711', 'Q920L2'), ('P02829', 'P34058'), ('P16140', 'P62815'), ('P41811', 'O35142'), ('P32492', 'P70569'), ('P41810', 'P23514'), ('P07251', 'P15999'), ('P36775', 'Q924S5'), ('P14742', 'P82808'), ('P02994', 'P62630'), ('P08964', 'Q9JLT0'), ('P00830', 'P10719'), ('P06738', 'P09811'), ('P38009', 'O35567'), ('P60010', 'P63259'), ('P02557', 'Q4QRB4'), ('P09733', 'Q5XIF6'), ('P12612', 'P28480'), ('P40413', 'Q68FQ0'), ('P09734', 'Q6AYZ1'), ('P32565', 'O88761'), ('P15108', 'P82995'), ('P39076', 'Q5XIM9'), ('P39077', 'Q6P502'), ('P33299', 'Q63347'), ('Q03940', 'P60123'), ('P12709', 'Q6P6V0'), ('P08417', 'P14408'), ('P18759'

In [56]:
for gotype in ["molecular_function", "biological_process", "cellular_component"]:
    fc = semantic_sim(pairs_, df1, df2,
                    obofile = "../../data/go/go-basic.obo",
                    annot1file=annot1,
                    annot2file = annot2,
                    type = gotype)
    print(f"\t For GO={gotype}, FC = {fc}")
    FC[f"FC-{gotype}(FINAL)"] = fc

  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=molecular_function, FC = 0.4939047018104258
  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=biological_process, FC = 0.22340937208497047
  EXISTS: ../../data/go/go-basic.obo
../../data/go/go-basic.obo: fmt(1.2) rel(2022-12-04) 46,763 Terms; optional_attrs(relationship)
	 For GO=cellular_component, FC = 0.41608408277642783
